# Introduction/Business Problem

<font size ="4">In this project, I would like to answer “how accurate is the Foursquare data “. The purpose of this project is provide a recommendation of proxy data collection platform for citizens or government to understand the neighborhood when official data is not available. For instance, how many bicycle shops are in the area of City of Toronto, Canada and which one is the closest to me.</font>

<h3> Data</h3>

<font size ="4">To get the solution to the above-discussed problem, we will require data that shows the bicycle shops location. Various fields required such as: name of the shop, address or location information such as longitude and latitude, their ratings, and other useful information. </font>

<font size ="4">The data source contains the necessary fields of interest are found from City of Toronto’s Open Data Portal(https://open.toronto.ca/dataset/bicycle-shops/) as well as the Foursquare data.</font>

In [2]:
import pandas as pd

In [72]:
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/e285b0d5-afbc-47e6-b6bf-2f74d9b5a9d7?format=csv&projection=4326"
df = pd.read_csv (url)
df
#drop unwanted columns
df.drop(['_id','ADDRESS_POINT_ID','ADDRESS_NUMBER','LINEAR_NAME_FULL','MUNICIPALITY','PLACE_NAME','GENERAL_USE_CODE','CENTRELINE_ID','LO_NUM','LO_NUM_SUF','HI_NUM','HI_NUM_SUF','LINEAR_NAME_ID','X','Y','ID','UNIT','OBJECTID'],axis =1,inplace =True)

<h3>Approach</h3>

1. Collect the Toronto bike shop data from https://open.toronto.ca/dataset/bicycle-shops/ as above mentioned.
2. Using FourSquare API we will find all venues for each neighborhood in Toronto.
3. Filter out all venues that are bike shop.
4. Find rating for each bike shop using FourSquare API.
5. Using rating for each resturant , we will sort that data.
6. Visualize the Ranking of neighborhoods using folium library(python)

     |████████████████████████████████| 92kB 23.0MB/s eta 0:00:01
Done.


<h3> Methodology</h3>

We will import the required libraries for python:
1. pandas and numpy for handling data.
2. request module for using FourSquare API.
3. geopy to get co-ordinates of City of Toronto
4. folium to visualize the results on a map

In [63]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
! pip install geocoder
import geocoder
! pip install folium
import folium # map rendering library
import matplotlib.pyplot as plt
import datetime
from pandas.io.json import json_normalize 

print('Done.')

Done.


In [73]:
# create folium map
map =folium.Map(location=[43.7181552,-79.5184855], zoom_start=11)
for lat, lng, name, addr,phone in zip(df['LATITUDE'], df['LONGITUDE'], df['NAME'] ,df['ADDRESS_FULL'],df['PHONE']):
    label = '{}, {} ,{}'.format(name, addr, phone)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([float(lat),float(lng)],
                       radius = 7,
                       popup = label,
                       color = 'blue',
                       fill = True,
                       fill_color = '#a72920',
                       fill_opacity = 0.5,
                       parse_html = False).add_to(scarboro_map)
map

In [58]:
now = datetime.datetime.now()
date = "%4d%02d%02d" % (now.year, now.month, now.day)
CLIENT_ID = 'TPTFTJWDADAUO13EZ1MUGBXXGLHS3YMKYWLLOXJ2QHO1MQK3' # Foursquare ID
CLIENT_SECRET = 'QUK42HCIKVO54GS5CZDHTAH1OHLQYPEYRKI3I05Z5FZTT0MQ' # Foursquare Secret
VERSION = date


In [76]:
i = 0
latitude = 43.7181552
longitude = -79.5184855
neighborhood_name = "Bike Shop"
radius = 1000
limit = 200
url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        latitude,
        longitude,
        VERSION,
        radius,
        limit)
results = requests.get(url).json()
venues = results['response']['venues']
nearby_venues = json_normalize(venues)

In [77]:
# create folium map
scarboro_map = folium.Map(location = [43.7181552, -79.5184855], zoom_start = 11
                        )
for lat, lng, name in zip(nearby_venues['location.lat'], nearby_venues['location.lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([float(lat),float(lng)],
                       radius = 3,
                       popup = label,
                       color = 'red',
                       fill = True,
                       fill_color = '#a72920',
                       fill_opacity = 0.5,
                       parse_html = False).add_to(scarboro_map)
display(scarboro_map)

<h3>Discussion</h3>

Just vistually visit two maps, the dots are almost allocated to different places. From the map using the data obtained from Open Data Potal from the City of Toronto, the bike shops are over the places, while in the second map the dots are focusing in the north area of the map. The difference might be bacause of the date of the data being updated. The data from the City of Toronto website were that of 2016, while Foursquare data are supposed to be up to dated. 

<h3> Conclusion</h3>

Just judging from the maps, the Foursquare data is not reliable. However, more testing might be needed to justify this. Also the data used to perform the test should be updated to current.

Thank you